In [63]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import numpy as np

In [47]:
data = pd.read_csv("E:\\Yelp\\Unfiltered Data\\YelpZip\\Customs\\SAMPLE", header=None)

In [48]:
data.head()

,0,1,2,3,4,5,6,7
0,0,0,5051,2014-12-05,Extremely slow kitchen went hour eat shift tol...,0,1.0,1
1,1,1,5071,2014-05-20,AWFUL half hour kill bypassed Capogiro go read...,0,1.0,1
2,2,2,5071,2014-05-20,Lovely spot Especially eat outside Food excell...,0,1.0,1
3,3,3,5076,2014-03-12,Sunday ordered soy caramel latte caramel vanil...,0,1.0,1
4,4,4,5077,2014-02-20,Toast youre killing Ive tried many times like ...,0,1.0,1


In [49]:
data = data[[4, 7]]

In [50]:
stop_words=set(stopwords.words("english"))
print("Stopwords from the NLTK corpus: ", stop_words)

def text_process(text):
    #String punctuation provides all the necessary checks
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

import tqdm
tqdm.tqdm.pandas()
data[4] = data[4].progress_apply(lambda txt: str(txt))
data[4] = data[4].progress_apply(lambda txt: text_process(txt))

Stopwords from the NLTK corpus:  {'now', 'have', 'against', 'we', 'above', "you'd", "shouldn't", 'doing', "that'll", 'very', "you've", 't', 'do', 'under', "it's", 'him', 'herself', 'to', 'i', 'and', 'from', 'any', 'because', 'during', 'ain', 'y', 'off', 'll', 'isn', 'being', 'was', 'weren', 'here', 'she', 'until', 'most', 'again', 'out', "wasn't", 'down', 'only', 'in', 'what', "needn't", 'don', "don't", "shan't", 'mustn', 're', 'with', 'myself', "she's", 'needn', 'whom', 'by', 'those', 'about', 'm', 'too', "mustn't", 'where', 'aren', 'ourselves', 'both', 'no', 'yourself', 'haven', 'as', "won't", 'at', 'there', 'were', 's', 'if', 'further', 'he', 'having', 'up', 'who', "weren't", 'your', 'on', 'own', 'after', 'does', 'how', 'some', 'hers', "haven't", 'am', "wouldn't", 'that', 'wasn', 'an', 'yours', 'couldn', 'nor', 'himself', 'd', 'each', "you'll", 'themselves', 'yourselves', 'into', "aren't", 'just', 'between', 'be', 'the', 'they', "hasn't", 'are', 'not', 've', 'didn', 'had', 'once', '


  3%|██                                                                            | 543/20000 [00:08<06:26, 50.32it/s]


  6%|████▌                                                                        | 1181/20000 [00:18<04:46, 65.64it/s]


 10%|███████▎                                                                     | 1903/20000 [00:29<03:28, 86.84it/s]


 12%|█████████▋                                                                   | 2500/20000 [00:40<07:55, 36.83it/s]


...

 91%|█████████████████████████████████████████████████████████████████████▏      | 18203/20000 [05:14<00:47, 37.47it/s]


 93%|██████████████████████████████████████████████████████████████████████▉     | 18672/20000 [05:24<00:26, 50.03it/s]


 96%|████████████████████████████████████████████████████████████████████████▉   | 19187/20000 [05:34<00:13, 60.35it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19666/20000 [05:43<00:09, 35.20it/s]


100%|████████████████████████████████████████████████████████████████████████████| 20000/20000 [05:50<00:00, 57.02it/s]

In [90]:
def binarize(n):
    if n==1:
        return 1
    else:
        return 0

data[7] = data[7].apply(lambda n: binarize(n))
data[7].head()

0    1
1    1
2    1
3    1
4    1
Name: 7, dtype: int64

In [66]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data[4])

sequences = tokenizer.texts_to_sequences(data[4])
data1 = pad_sequences(sequences, maxlen=60)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1, data[7], test_size=0.25)

# Simple RNN

In [70]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=60))
model.add(LSTM(500, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(100, dropout=0.2, recurrent_dropout=0.2))
# model.add(Dense(10, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 60, 100)           2000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 500)               1202000   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 501       
Total params: 3,202,501
Trainable params: 3,202,501
Non-trainable params: 0
_________________________________________________________________


In [74]:
model.fit(data1, np.array(data[7]), validation_split=0.25, batch_size = 1000, epochs=3)

Train on 15000 samples, validate on 5000 samples
Epoch 1/3
15000/15000 [==============================] - ETA: 1:14 - loss: 0.5100 - acc: 0.734 - ETA: 1:08 - loss: 0.4972 - acc: 0.751 - ETA: 1:05 - loss: 0.5019 - acc: 0.749 - ETA: 1:01 - loss: 0.4960 - acc: 0.751 - ETA: 57s - loss: 0.4957 - acc: 0.749 - ETA: 52s - loss: 0.4950 - acc: 0.75 - ETA: 46s - loss: 0.4900 - acc: 0.75 - ETA: 40s - loss: 0.4898 - acc: 0.75 - ETA: 34s - loss: 0.4890 - acc: 0.75 - ETA: 28s - loss: 0.4881 - acc: 0.75 - ETA: 22s - loss: 0.4871 - acc: 0.75 - ETA: 16s - loss: 0.4870 - acc: 0.75 - ETA: 11s - loss: 0.4855 - acc: 0.76 - ETA: 5s - loss: 0.4855 - acc: 0.7622 - 94s 6ms/step - loss: 0.4849 - acc: 0.7627 - val_loss: 1.1224 - val_acc: 0.3050
Epoch 2/3
15000/15000 [==============================] - ETA: 1:16 - loss: 0.4515 - acc: 0.779 - ETA: 1:09 - loss: 0.4546 - acc: 0.785 - ETA: 1:03 - loss: 0.4558 - acc: 0.785 - ETA: 58s - loss: 0.4592 - acc: 0.787 - ETA: 52s - loss: 0.4556 - acc: 0.79 - ETA: 47s - loss: 0.

In [80]:
scores = model.evaluate(X_test, y_test)

4000/4000 [==============================] - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  

In [83]:
print("Accuracy: ", scores[1]*100, "%")

Accuracy:  72.52499999999999 %


# Deep RNN 

In [85]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=60))
model.add(LSTM(500, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(100))
model.add(Dropout(0.2))
model.add(Dense(10))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 60, 100)           2000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 500)               1202000   
_________________________________________________________________
dense_4 (Dense)              (None, 100)               50100     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
_________________________________________________________________
dropout_2 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total para

In [86]:
model.fit(data1, np.array(data[7]), validation_split=0.25, batch_size = 1000, epochs=3)

Train on 15000 samples, validate on 5000 samples
Epoch 1/3
15000/15000 [==============================] - ETA: 1:42 - loss: 0.6929 - acc: 0.530 - ETA: 1:22 - loss: 0.6888 - acc: 0.598 - ETA: 1:12 - loss: 0.6832 - acc: 0.620 - ETA: 1:04 - loss: 0.6727 - acc: 0.636 - ETA: 57s - loss: 0.7212 - acc: 0.641 - ETA: 51s - loss: 0.7099 - acc: 0.64 - ETA: 44s - loss: 0.7029 - acc: 0.64 - ETA: 39s - loss: 0.6976 - acc: 0.64 - ETA: 33s - loss: 0.6938 - acc: 0.64 - ETA: 27s - loss: 0.6904 - acc: 0.65 - ETA: 21s - loss: 0.6884 - acc: 0.65 - ETA: 16s - loss: 0.6865 - acc: 0.65 - ETA: 10s - loss: 0.6846 - acc: 0.65 - ETA: 5s - loss: 0.6829 - acc: 0.6568 - 91s 6ms/step - loss: 0.6812 - acc: 0.6579 - val_loss: 0.8296 - val_acc: 0.0000e+00
Epoch 2/3
15000/15000 [==============================] - ETA: 1:12 - loss: 0.6563 - acc: 0.663 - ETA: 1:11 - loss: 0.6533 - acc: 0.670 - ETA: 1:12 - loss: 0.6525 - acc: 0.668 - ETA: 1:07 - loss: 0.6491 - acc: 0.672 - ETA: 1:02 - loss: 0.6456 - acc: 0.675 - ETA: 57s - l

In [88]:
scores = model.evaluate(X_test, y_test)

4000/4000 [==============================] - ETA: 17 - ETA: 16 - ETA: 15 - ETA: 15 - ETA: 15 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 14 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 13 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 12 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 11 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 10 - ETA: 9 - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 

In [89]:
print("Accuracy: ", scores[1]*100, "%")

Accuracy:  58.95 %


# Accuracy on a deeper network didn't turn out to be better.
# Training the 1st model based on Bigrams

In [94]:
data = pd.read_csv("E:\\Yelp\\Unfiltered Data\\YelpZip\\Customs\\SAMPLE", header=None)

In [97]:
from nltk.corpus import stopwords
import string
stop_words=set(stopwords.words("english"))
print("Stopwords from the NLTK corpus: ", stop_words)

def text_process(text):
    #String punctuation provides all the necessary checks
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

import tqdm
tqdm.tqdm.pandas()
data[4] = data[4].progress_apply(lambda txt: str(txt))
data[4] = data[4].progress_apply(lambda txt: text_process(txt))

Stopwords from the NLTK corpus:  {'now', 'have', 'against', 'we', 'above', "you'd", "shouldn't", 'doing', "that'll", 'very', "you've", 't', 'do', 'under', "it's", 'him', 'herself', 'to', 'i', 'and', 'from', 'any', 'because', 'during', 'ain', 'y', 'off', 'll', 'isn', 'being', 'was', 'weren', 'here', 'she', 'until', 'most', 'again', 'out', "wasn't", 'down', 'only', 'in', 'what', "needn't", 'don', "don't", "shan't", 'mustn', 're', 'with', 'myself', "she's", 'needn', 'whom', 'by', 'those', 'about', 'm', 'too', "mustn't", 'where', 'aren', 'ourselves', 'both', 'no', 'yourself', 'haven', 'as', "won't", 'at', 'there', 'were', 's', 'if', 'further', 'he', 'having', 'up', 'who', "weren't", 'your', 'on', 'own', 'after', 'does', 'how', 'some', 'hers', "haven't", 'am', "wouldn't", 'that', 'wasn', 'an', 'yours', 'couldn', 'nor', 'himself', 'd', 'each', "you'll", 'themselves', 'yourselves', 'into', "aren't", 'just', 'between', 'be', 'the', 'they', "hasn't", 'are', 'not', 've', 'didn', 'had', 'once', '




  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]


100%|████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 426588.56it/s]


  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]


  0%|                                                                                | 9/20000 [00:00<04:02, 82.29it/s]


  0%|                                                                               | 16/20000 [00:00<04:23, 75.80it/s]


  0%|                                                                               | 23/20000 [00:00<04:38, 71.83it/s]


  0%|                                                                               | 30/20000 [00:00<05:01, 66.22it/s]


  0%|▏                                                                              | 35/20000 [00:00<06:56, 47.89it/s]


  0%|▏               

  3%|██▍                                                                           | 640/20000 [00:08<04:29, 71.91it/s]


  3%|██▌                                                                           | 649/20000 [00:08<05:19, 60.59it/s]


  3%|██▌                                                                           | 657/20000 [00:09<05:25, 59.44it/s]


  3%|██▌                                                                           | 669/20000 [00:09<04:40, 68.91it/s]


  3%|██▋                                                                           | 680/20000 [00:09<04:13, 76.10it/s]


  3%|██▋                                                                           | 693/20000 [00:09<03:46, 85.31it/s]


  4%|██▊                                                                           | 707/20000 [00:09<03:23, 94.80it/s]


  4%|██▊                                                                           | 718/20000 [00:09<05:06, 62.88it/s]


  4%|██▊                

  6%|████▊                                                                        | 1235/20000 [00:17<04:29, 69.70it/s]


  6%|████▊                                                                        | 1243/20000 [00:17<04:25, 70.70it/s]


  6%|████▊                                                                        | 1251/20000 [00:17<04:44, 65.91it/s]


  6%|████▊                                                                        | 1260/20000 [00:17<04:37, 67.63it/s]


  6%|████▉                                                                        | 1268/20000 [00:18<04:41, 66.50it/s]


  6%|████▉                                                                        | 1275/20000 [00:18<05:22, 58.07it/s]


  6%|████▉                                                                        | 1282/20000 [00:18<05:38, 55.31it/s]


  6%|████▉                                                                        | 1291/20000 [00:18<05:14, 59.45it/s]


  7%|█████              

 10%|███████▍                                                                     | 1940/20000 [00:28<06:24, 47.02it/s]


 10%|███████▌                                                                     | 1949/20000 [00:28<05:34, 53.96it/s]


 10%|███████▌                                                                     | 1956/20000 [00:29<05:54, 50.85it/s]


 10%|███████▌                                                                     | 1962/20000 [00:29<06:01, 49.96it/s]


 10%|███████▌                                                                     | 1973/20000 [00:29<05:06, 58.85it/s]


 10%|███████▋                                                                     | 1983/20000 [00:29<04:33, 65.90it/s]


 10%|███████▋                                                                     | 1991/20000 [00:29<05:28, 54.83it/s]


 10%|███████▋                                                                     | 1998/20000 [00:29<05:38, 53.21it/s]


 10%|███████▋           

...

 89%|███████████████████████████████████████████████████████████████████▎        | 17704/20000 [05:09<00:45, 50.36it/s]


 89%|███████████████████████████████████████████████████████████████████▎        | 17713/20000 [05:09<00:40, 57.00it/s]


 89%|███████████████████████████████████████████████████████████████████▎        | 17727/20000 [05:09<00:34, 66.85it/s]


 89%|███████████████████████████████████████████████████████████████████▍        | 17735/20000 [05:10<00:39, 57.14it/s]


 89%|███████████████████████████████████████████████████████████████████▍        | 17742/20000 [05:10<00:45, 49.30it/s]


 89%|███████████████████████████████████████████████████████████████████▍        | 17748/20000 [05:10<01:12, 31.09it/s]


 89%|███████████████████████████████████████████████████████████████████▍        | 17754/20000 [05:10<01:02, 35.74it/s]


 89%|███████████████████████████████████████████████████████████████████▍        | 17760/20000 [05:10<00:56, 39.91it/s]


 89%|███████████████████

 91%|█████████████████████████████████████████████████████████████████████       | 18158/20000 [05:18<00:37, 48.77it/s]


 91%|█████████████████████████████████████████████████████████████████████       | 18165/20000 [05:18<00:34, 52.52it/s]


 91%|█████████████████████████████████████████████████████████████████████       | 18171/20000 [05:19<00:35, 51.08it/s]


 91%|█████████████████████████████████████████████████████████████████████       | 18177/20000 [05:19<00:39, 46.48it/s]


 91%|█████████████████████████████████████████████████████████████████████       | 18187/20000 [05:19<00:33, 54.52it/s]


 91%|█████████████████████████████████████████████████████████████████████▏      | 18194/20000 [05:19<00:37, 47.91it/s]


 91%|█████████████████████████████████████████████████████████████████████▏      | 18200/20000 [05:19<00:37, 47.94it/s]


 91%|█████████████████████████████████████████████████████████████████████▏      | 18206/20000 [05:19<00:40, 44.62it/s]


 91%|███████████████████

 93%|██████████████████████████████████████████████████████████████████████▋     | 18597/20000 [05:28<00:29, 47.60it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 18603/20000 [05:28<00:31, 44.41it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 18610/20000 [05:28<00:28, 48.90it/s]


 93%|██████████████████████████████████████████████████████████████████████▋     | 18616/20000 [05:28<00:27, 50.55it/s]


 93%|██████████████████████████████████████████████████████████████████████▊     | 18622/20000 [05:28<00:27, 49.76it/s]


 93%|██████████████████████████████████████████████████████████████████████▊     | 18628/20000 [05:28<00:30, 45.71it/s]


 93%|██████████████████████████████████████████████████████████████████████▊     | 18633/20000 [05:28<00:32, 42.10it/s]


 93%|██████████████████████████████████████████████████████████████████████▊     | 18638/20000 [05:29<00:34, 39.90it/s]


 93%|███████████████████

 95%|████████████████████████████████████████████████████████████████████████▌   | 19096/20000 [05:37<00:19, 45.54it/s]


 96%|████████████████████████████████████████████████████████████████████████▌   | 19102/20000 [05:37<00:20, 43.14it/s]


 96%|████████████████████████████████████████████████████████████████████████▌   | 19108/20000 [05:37<00:19, 46.09it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 19116/20000 [05:38<00:17, 50.32it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 19122/20000 [05:38<00:17, 49.60it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 19131/20000 [05:38<00:16, 53.20it/s]


 96%|████████████████████████████████████████████████████████████████████████▋   | 19137/20000 [05:38<00:16, 51.52it/s]


 96%|████████████████████████████████████████████████████████████████████████▊   | 19149/20000 [05:38<00:13, 61.28it/s]


 96%|███████████████████

 98%|██████████████████████████████████████████████████████████████████████████▌ | 19634/20000 [05:46<00:06, 58.18it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19641/20000 [05:46<00:07, 48.23it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19647/20000 [05:46<00:08, 41.87it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19656/20000 [05:46<00:07, 46.71it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19663/20000 [05:47<00:06, 50.84it/s]


 98%|██████████████████████████████████████████████████████████████████████████▋ | 19669/20000 [05:47<00:06, 49.95it/s]


 98%|██████████████████████████████████████████████████████████████████████████▊ | 19675/20000 [05:47<00:06, 51.33it/s]


 98%|██████████████████████████████████████████████████████████████████████████▊ | 19682/20000 [05:47<00:05, 54.58it/s]


 98%|███████████████████

In [98]:
data[7] = data[7].apply(lambda n: binarize(n))
data.head()

,0,1,2,3,4,5,6,7
0,0,0,5051,2014-12-05,"[Extremely, slow, kitchen, went, hour, eat, sh...",0,1.0,1
1,1,1,5071,2014-05-20,"[AWFUL, half, hour, kill, bypassed, Capogiro, ...",0,1.0,1
2,2,2,5071,2014-05-20,"[Lovely, spot, Especially, eat, outside, Food,...",0,1.0,1
3,3,3,5076,2014-03-12,"[Sunday, ordered, soy, caramel, latte, caramel...",0,1.0,1
4,4,4,5077,2014-02-20,"[Toast, youre, killing, Ive, tried, many, time...",0,1.0,1


In [100]:
def join_back(txt):
    new = ''
    for x in txt:
        new += ''.join(x)+" "
    new = new[:-1]
    return new

data[4] = data[4].progress_apply(lambda tokens: join_back(tokens))




  0%|                                                                                        | 0/20000 [00:00<?, ?it/s]


 15%|███████████                                                               | 2975/20000 [00:00<00:00, 27198.70it/s]


 32%|███████████████████████▉                                                  | 6484/20000 [00:00<00:00, 28502.03it/s]


 50%|████████████████████████████████████▊                                     | 9950/20000 [00:00<00:00, 29390.63it/s]


 70%|███████████████████████████████████████████████████▎                     | 14043/20000 [00:00<00:00, 31416.01it/s]


 89%|████████████████████████████████████████████████████████████████▋        | 17719/20000 [00:00<00:00, 32044.32it/s]


100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 32005.86it/s]

In [147]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range = (2,2))
data1 = vect.fit_transform(data[4])
print(data1.shape)

(20000, 611148)


In [106]:
model = Sequential()
model.add(Embedding(20000, 100, input_length=trainX.shape[1]))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 611148, 100)       2000000   
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 2,080,501
Trainable params: 2,080,501
Non-trainable params: 0
_________________________________________________________________


In [108]:
model.fit(data1, np.array(data[7]), validation_split=0.25, batch_size = 1000, epochs=5)

Train on 15000 samples, validate on 5000 samples
Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[1000,611148,100] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[Node: embedding_6/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_2/Adam/Assign_2"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_6/embeddings/read, embedding_6/Cast, training_2/Adam/gradients/embedding_6/embedding_lookup_grad/concat/axis)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
scores = model.evaluate(X_test, y_test)

In [ ]:
print("Accuracy: ", scores[1]*100, "%")

611148

In [112]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
 
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer
 
# calculate the maximum document length
def max_length(lines):
    return max([len(s.split()) for s in lines])
 
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded
 
# define the model
def define_model(length, vocab_size):
	# channel 1
	inputs1 = Input(shape=(length,))
	embedding1 = Embedding(vocab_size, 100)(inputs1)
	conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
	drop1 = Dropout(0.5)(conv1)
	pool1 = MaxPooling1D(pool_size=2)(drop1)
	flat1 = Flatten()(pool1)
	# channel 2
	inputs2 = Input(shape=(length,))
	embedding2 = Embedding(vocab_size, 100)(inputs2)
	conv2 = Conv1D(filters=32, kernel_size=6, activation='relu')(embedding2)
	drop2 = Dropout(0.5)(conv2)
	pool2 = MaxPooling1D(pool_size=2)(drop2)
	flat2 = Flatten()(pool2)
	# channel 3
	inputs3 = Input(shape=(length,))
	embedding3 = Embedding(vocab_size, 100)(inputs3)
	conv3 = Conv1D(filters=32, kernel_size=8, activation='relu')(embedding3)
	drop3 = Dropout(0.5)(conv3)
	pool3 = MaxPooling1D(pool_size=2)(drop3)
	flat3 = Flatten()(pool3)
	# merge
	merged = concatenate([flat1, flat2, flat3])
	# interpretation
	dense1 = Dense(10, activation='relu')(merged)
	outputs = Dense(1, activation='sigmoid')(dense1)
	model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
	# compile
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	# summarize
	print(model.summary())
	return model

data = pd.read_csv("E:\\Yelp\\Unfiltered Data\\YelpZip\\Customs\\SAMPLE", header=None)
data[4] = data[4].apply(lambda txt: str(txt))
# load training dataset
trainLines = data[4]
data[7] = data[7].apply(lambda n: binarize(n))
trainLabels = data[7] 
# create tokenizer
tokenizer = create_tokenizer(trainLines)
# calculate max document length
length = max_length(trainLines)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, trainLines, length)
print(trainX.shape)
 
# define model
model = define_model(length, vocab_size)
# fit model
model.fit([trainX,trainX,trainX], array(trainLabels), epochs=3, batch_size=500)
# save the model
model.save('model.h5')

Max document length: 507
Vocabulary size: 43499
(20000, 507)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 507)          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 507)          0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, 507)          0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 507, 100)     4349900     input_7[0][0]                    
________________________________________________

KeyboardInterrupt: 